In [12]:
!pip install transformers datasets rouge-score

import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from rouge_score import rouge_scorer
import numpy as np
import os
from tqdm import tqdm

In [13]:
df = pd.read_parquet("Khmer_Data.parquet")
df = df.sample(n=10000, random_state=42)  # subset 3000 samples
dataset = Dataset.from_pandas(df)

In [14]:
# Step 2: Train / Validation / Test split
# ------------------------------
train_test = dataset.train_test_split(test_size=0.2, seed=42)
val_test = train_test['test'].train_test_split(test_size=0.5, seed=42)

train_data = train_test['train']
val_data   = val_test['train']
test_data  = val_test['test']

print("Train:", len(train_data), "Val:", len(val_data), "Test:", len(test_data))

Train: 8000 Val: 1000 Test: 1000


In [15]:
# Step 3: Load mT5 model & tokenizer
# ------------------------------
model_name = "google/mt5-small"  # small → CPU-friendly
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


In [16]:
# Step 4: Preprocessing
# ------------------------------
max_input_len = 512
max_output_len = 128

def preprocess(batch):
    inputs = tokenizer(batch['full_article'], truncation=True, padding='max_length', max_length=max_input_len)
    labels = tokenizer(batch['summary'], truncation=True, padding='max_length', max_length=max_output_len)
    batch['input_ids'] = inputs['input_ids']
    batch['attention_mask'] = inputs['attention_mask']
    batch['labels'] = labels['input_ids']
    return batch

train_data = train_data.map(preprocess)
val_data = val_data.map(preprocess)
test_data = test_data.map(preprocess)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
# Step 5: Collate function for DataLoader
# ------------------------------
def batch_pad(batch):
    input_ids = pad_sequence([torch.tensor(x['input_ids']) for x in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([torch.tensor(x['attention_mask']) for x in batch], batch_first=True, padding_value=0)
    labels = pad_sequence([torch.tensor(x['labels']) for x in batch], batch_first=True, padding_value=-100)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

train_loader = DataLoader(train_data, batch_size=4, shuffle=True, collate_fn=batch_pad)
val_loader   = DataLoader(val_data, batch_size=4, collate_fn=batch_pad)


In [18]:
# Step 6: Training setup
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

num_epochs = 15
patience = 3
best_val_loss = float('inf')
epochs_no_improve = 0

MODEL_DIR = "/content/drive/MyDrive/mt5_khmer_model"
os.makedirs(MODEL_DIR, exist_ok=True)

# ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=False)

In [19]:
# Step 7: Training loop with Early Stopping
# ------------------------------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs} -> Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # Early Stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        # Save best model
        model.save_pretrained(MODEL_DIR)
        tokenizer.save_pretrained(MODEL_DIR)
        print("Validation improved. Model saved.")
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epoch(s).")
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

Epoch 1: 100%|██████████| 2000/2000 [14:17<00:00,  2.33it/s]


Epoch 1/15 -> Train Loss: 10.9678, Val Loss: 4.7182
Validation improved. Model saved.


Epoch 2: 100%|██████████| 2000/2000 [14:12<00:00,  2.35it/s]


Epoch 2/15 -> Train Loss: 4.5779, Val Loss: 2.6816
Validation improved. Model saved.


Epoch 3: 100%|██████████| 2000/2000 [14:13<00:00,  2.34it/s]


...

Epoch 13/15 -> Train Loss: 1.7517, Val Loss: 1.5293
Validation improved. Model saved.


Epoch 14: 100%|██████████| 2000/2000 [14:13<00:00,  2.34it/s]


Epoch 14/15 -> Train Loss: 1.7167, Val Loss: 1.5111
Validation improved. Model saved.


Epoch 15: 100%|██████████| 2000/2000 [14:13<00:00,  2.34it/s]


Epoch 15/15 -> Train Loss: 1.6820, Val Loss: 1.5100
Validation improved. Model saved.


In [23]:
# Step 8: Evaluate first 10 validation articles
# ------------------------------
model.eval()
test_samples = val_data.select(range(20))
test_texts = [x['full_article'] for x in test_samples]
test_refs = [x['summary'] for x in test_samples]

inputs = tokenizer(test_texts, return_tensors="pt", truncation=True, padding=True, max_length=max_input_len).to(device)
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_length=max_output_len, num_beams=4, early_stopping=True)

pred_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Print full article + reference + prediction + ROUGE
for i, (article, ref, pred) in enumerate(zip(test_texts, test_refs, pred_texts)):
    score = scorer.score(ref, pred)
    print(f"\n--- Article {i+1} ---")
    print("Full Article:\n", article)
    print("\nReference Summary:\n", ref)
    print("\nPredicted Summary:\n", pred)
    print("ROUGE-1:", round(score['rouge1'].fmeasure,4))
    print("ROUGE-2:", round(score['rouge2'].fmeasure,4))
    print("ROUGE-L:", round(score['rougeL'].fmeasure,4))
    print("-"*100)


--- Article 1 ---
Full Article:
 Sid Ahmed Ghlam អាយុ២៤ឆ្នាំ ជា​និស្សិត​រៀន​ផ្នែក​អេឡិចត្រូនិក នៅ​បារាំង ឧស្សាហ៍​ឡើង​ចុះ​ប្រទេស​បារាំង ព្រោះ​មានគ្រួសារ​រស់នៅ​ទីនោះ។ ជនសង្ស័យ​រូបនេះ​ស្គាល់​មនុស្សម្នា​ក់នៅ​ស៊ី​រី ដែល​បាន​សុំ​គេ​អោយ​វាយប្រហារ​វិហារ​គ្រិស្តសាសនា​មួ​ួ​យ រឺក៏​ពីរ​កន្លែង ក្នុង​ទីក្រុង​ប៉ារីស។ ការចាប់ខ្លួន​Sid Ahmed Ghlamមិនបាន​កើតឡើង​ដោយ​ព្រាងទុក​មុន​ឡើយ។ កាលពី​ព្រឹក​ថ្ងៃអាទិត្យ ជនសង្ស័យ​រូបនេះ​បាន​ទូរស័ព្ទ​ទៅ ក្រុម​SAMU ប្រាប់ថា ខ្លួន​របួស​ដោយ​គ្រាប់កាំភ្លើង ក្នុង​ហេតុការណ៍​លួច​មួយ នៅក្នុង​សង្កា​ត់ទី១៣ ក្នុង​ក្រុងប៉ារីស។ ក្រុម​SAMU បាន​រាយការណ៍​ទៅ​ប៉ូលិស។ ប៉ូលិស​ក៏បាន​រកឃើញ​អាវុធ​ជាច្រើន និង​អាវ​ការពារ​គ្រាប់ នៅក្នុង​រថយន្ត​របស់​ជនសង្ស័យ។ ក្រុម​អង្កេត​កំពុង​ស៊ើប​ថា តើ​Sid Ahmed Ghlam បាន​អាវុធ​ទាំងនេះ​មកពីណា។ នៅផ្ទះ​របស់​Sid Ahmed Ghlam ប៉ូលិស​បាន​រកឃើញ​ឯកសារ​ជាច្រើន ស្តីពី​ក្រុម​អាល់​កៃ​ដា និង​ក្រុម​ជី​ហាត​រដ្ឋ​អ៊ីស្លាម។ ប៉ូលិស​បារាំង​ក៏បាន​ចាប់ខ្លួន​ស្ត្រី​សង្ស័យ​ម្នាក់​ដែល​ធ្លាប់​ស្គាល់​និស្សិត​រូបនេះ​ដែរ។ ផ្នែក​ចារកិច្ច​បារាំង​ធ្លាប់បាន​កត់សំគាល់​ជនល្មើស​រួច​ម្តង​ហើយ ដោ

In [21]:
# Your Khmer text
text = "ប្រទេស​​ចិន​បានដើរ​តួនាទី​​ជា​មជ្ឈមណ្ឌល​​ផ្គត់ផ្គង់​ទំនិញ​កាន់​តែ​​​សំខាន់​សម្រាប់ប្រទេស​កម្ពុជា ដោយ​ក្នុង​ឆ្នាំ​២០២៥ កន្លង​មក​ទំនិញ​ដែល​ប្រទេស​ចិន​នាំចូលកម្ពុជា​ មាន​ទឹក​ប្រាក់​​ជាង ១៨​ ពាន់​លាន​ដុល្លារ កើន​ជាង ៣០​% ​បើ​ធៀប​នឹង​ឆ្នាំ​២០២៤។ ប្រាក់​ដែល​កម្ពុជា​​បញ្ចេញ​សម្រាប់នាំចូល​​​ទំនិញ​ពី​ប្រទេសចិន​បាន​គ្រប​ដណ្តប់​លើស ៥០% នៃ​ចំនួន​ទឹក​ប្រាក់​សរុប​ដែល​កម្ពុជា​​បញ្ចេញ​សម្រាប់​​នាំ​ចូល​ទំនិញ​ពី​ប្រទេសដៃគូ​ពាណិជ្ជកម្ម​ទាំង​អស់​។​យោង​តាម​របាយការណ៍​របស់​​​អគ្គនាយក​ដ្ឋាន​គយ និង​រដ្ឋាករ​កម្ពុជា​(GDCE) ក្នុង​ឆ្នាំ ២០២៥ កម្ពុជា​បាន​បញ្ចេញ​ប្រាក់​សរុប​ ៣៣,៨៨ ពាន់​លាន​ដុល្លារ​ទៅ​លើ​ការ​នាំចូលទំនិញ​ពី​គ្រប់​ប្រទេស​ទាំង​អស់​លើ​ពិភពលោក ដោយ​កើន​ឡើង​ ១៨,៧%​ បើ​ធៀប​នឹង​ឆ្នាំ​២០២៤ ដែល​មាន ២៨,៥៤ ពាន់​លាន​ដុល្លារ​។ ក្នុងនោះ​​​ទំនិញ​នាំចូល​ពី​ប្រទេស​ចិន​មាន​ទឹក​ប្រាក់ ១៨,០៤ ពាន់លាន​ដុល្លារ កើន​ឡើង ៣៤,៣% ពី​ចំនួន ១៣,៤៣ ពាន់​លាន​ដុល្លារ​។ ទឹក​ប្រាក់ទិញ​ទំនិញ​ពី​ប្រទេស​ចិន​ស្មើ​នឹង ៥៣,២៥% នៃ​ចំនួន​ទឹក​ប្រាក់សរុប​ដែល​កម្ពុជា​បញ្ចេញ​សម្រាប់​​នាំ​ចូល​ទំនិញ​ពី​គ្រប់​ប្រទេស​ទាំង​អស់​។​"

# Tokenize and move to device
inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to(device)

# Generate summary
with torch.no_grad():
    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        early_stopping=True
    )

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Predicted Summary:\n", summary)


Predicted Summary:
 ទំនិញ នាំចូល ពី ប្រទេស ចិន បាន កើន ឡើង ១៨,៧% បើ ធៀប នឹង ឆ្នាំ ២០២៥ ដែល មាន ២៨,៥៤ ពាន់ លាន ដុល្លារ ។ នេះ បើ យោងតាម របាយការណ៍ របស់ អគ្គនាយក ដ្ឋាន គយ និង រដ្ឋាករ កម្ពុជា (GDCE) ។


In [22]:
# Your Khmer text
text = "ក្រុមអ្នកសង្កេតការណ៍អាស៊ាន (AOT) ចុះសង្កេតការណ៍ ផ្ទៀងផ្ទាត់ និងរាយការណ៍​អំពីស្ថានភាពនៅ​ភូមិកររបស់ប្រជាជន ក្នុងឃុំស្រអែម ស្រុកជាំក្សាន្ត ខេត្តព្រះវិហារ ព្រមទាំងទៅកាន់តំបន់ប្រាសាទព្រះវិហារ ដែលជាសម្បត្តិបេតិកភណ្ឌពិភពលោកដ៏មានតម្លៃបំផុតរបស់មនុស្សជាតិ។ការ​ចុះ​សង្កេតការណ៍​និង​ផ្ទៀង​ផ្ទាត់​នេះ គឺ​ជាការ​ចុះ​​លើក​ទី៦​ ទៅដល់​ទីតាំង​ដែល​រង​ការ​ខូច​ខាត​ពី​ការ​វាយប្រហារ​របស់​យោធាថៃ នៅ​ក្នុង​ខែ​មករា​នេះ ក្រោយ​ពេល​ប្រទេស​ទាំង​ពីរ បាន​ឈាន​ដល់​បទ​ឈប់​បាញ់​នៅថ្ងៃទី២៧ ខែធ្នូ ឆ្នាំ២០២៥។លោកស្រី ម៉ាលី សុជាតា អ្នក​នាំពាក្យ​ក្រសួង​ការពារជាតិ ថ្លែង​ថា កម្ពុជាតែងតែគោរព និងអនុវត្តយ៉ាងខ្ជាប់ខ្ជួននូវបទឈប់បាញ់​និង​សេចក្តីថ្លែងការណ៍រួមស្តីពីកិច្ចព្រមព្រៀងសន្តិភាពរវាងកម្ពុជានិងថៃ ព្រមទាំងកិច្ចព្រមព្រៀងពាក់ព័ន្ធដ៏ទៃទៀត។"
# Tokenize and move to device
inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to(device)

# Generate summary
with torch.no_grad():
    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        early_stopping=True
    )

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Predicted Summary:\n", summary)


Predicted Summary:
 លោកស្រី ម៉ាលី សុជាតា អ្នកសង្កេតការណ៍អាស៊ាន (AOT) ចុះ សង្កេតការណ៍ ផ្ទៀងផ្ទាត់ និងរាយការណ៍អំពីស្ថានភាពនៅ ភូមិកររបស់ប្រជាជន ក្នុងឃុំស្រអែម ស្រុកជាំក្សាន្ត ខេត្តព្រះវិហារ ដែល រង ការ ខូច ខាត ពី ការ វាយប្រហារ របស់ យោធាថៃ នៅថ្ងៃទី២៧ ខែធ្នូ ឆ្នាំ២០២៥។
